# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ

(405968, 6)
(227032, 5)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227032 entries, 0 to 11736
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    227032 non-null  object
 1   from    227032 non-null  object
 2   to      227032 non-null  object
 3   header  218946 non-null  object
 4   body    227032 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


# Data Preprocessing - cleaning

In [1]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv", nrows=1000)
df.head()

Mounted at /content/drive


,date,from,to,header,body
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


In [2]:
# nbre d'expéditeurs distincts
len(df['from'].unique())

149

In [3]:
# nbre de destinataires distincts
len(df['to'].unique())

231

In [4]:
import numpy as np
df = df.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]
df_emails = df
df_emails['NER_header'] = ''
df_emails['NER_body'] = ''
df_emails['body_dict'] = ''
df_emails['summary'] = ''
df_emails.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        1000 non-null   object
 1   from        1000 non-null   object
 2   to          1000 non-null   object
 3   header      784 non-null    object
 4   body        1000 non-null   object
 5   NER_header  1000 non-null   object
 6   NER_body    1000 non-null   object
 7   body_dict   1000 non-null   object
 8   summary     1000 non-null   object
dtypes: object(9)
memory usage: 70.4+ KB


In [5]:
df_emails.columns

Index(['date', 'from', 'to', 'header', 'body', 'NER_header', 'NER_body',
       'body_dict', 'summary'],
      dtype='object')

In [6]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

In [7]:
# !pip install -U spacy
#!python -m spacy download en_core_web_sm




In [8]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)

{"'s", 'a', 'anywhere', 'been', 'across', 'former', 'whenever', 'over', 'off', 'on', 'whence', 'made', 'to', 'very', 'can', 'somehow', 'everywhere', 'third', 'full', 'here', 'once', 'call', 'might', 'regarding', '‘ve', 'against', 'several', 'two', 'us', 'your', 'many', 'no', 'sixty', 'still', 'indeed', 'among', 'nevertheless', 'such', 'seeming', 'whose', 'see', 'nobody', 'every', 'would', "'ve", 'almost', 'because', 'least', 'before', 'formerly', 'doing', 'afterwards', 'further', 'hereafter', 'up', 'mostly', 'those', 'latter', 'whole', "n't", '’m', 'with', 'than', 'most', 'anything', 'him', 'through', 'as', 'ours', 'elsewhere', 'these', 'something', 'there', 'twenty', 'next', 'had', 'though', 'fifteen', 'neither', 'therefore', 'herein', 'although', 'hence', 'should', 'my', 'unless', 'done', 'along', 'after', 'both', 'otherwise', 'besides', 'onto', 'i', 'ever', 'yours', 'more', 'become', 'less', 'twelve', 'five', 'per', 'some', "'d", 'it', 'seem', 'below', 'by', "'re", 'our', 'they', 'g

In [9]:
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def untokenize (tokens):
# Suppression des caractres spéciaux ou inutiles \"',]
  r = re.compile(r"[\\\''\",]")
  texte=r.sub('', tokens)
  texte = sent_tokenize(texte)
#  print("Après :\n",type(texte)," ",texte)
  return texte

In [10]:
def remove_stop_words(text,stopwords):
  lst=[]
  for token in str(text).split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
  # Join items in the list
#  print("Original text  : ",text)
  result = ' '.join(lst)
#  print("Text after removing stopwords  :   ",result)
  return result

In [11]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [12]:
df_emails.shape[0]

1000

In [13]:
# test unitaire de la fonction dictionarize sur une ligne
#
# artikle = dictionarize(df_emails['body'][32])
# for cle, valeur in artikle.items():
#        print("clé", cle, "vaut", valeur)

In [14]:
import pandas as pd
import numpy as np


def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
#    print(summary)
    return summary

In [ ]:
# Nettoyage : Détokenisation et retokenisation en sentences
deb = 0
fin = df_emails.shape[0]
# fin = 100
for id in range(deb,fin):
#  print("[",id," :")
  text = df_emails['body'].iloc[id]
  text = untokenize(text)
#  text = remove_stop_words(text,stopwords)
  text = dictionarize(str(text))
  df_emails['body_dict'][id] = text
# résumé dans le champ 'summarize' en 3 lignes au plus.
  df_emails['summary'][id] = summarize(id, 3, df_emails)
#  print("apres remove stopwords() : ",df_emails['body_dict'][id])


In [ ]:
df_emails['body'][32]

In [ ]:
df_emails['summary'][32]

In [ ]:
# texte fonction de résumé sur la ligne 'id'
id = 12
#print(df_emails['body'][id])
for cle, valeur in df_emails['body_dict'][id].items():
        print("Ligne ", cle, "vaut", valeur)


print('summary :\n\n',summarize(id, 3, df_emails))

# df_emails['summary'].iloc[id] = summarize(id, 5, df_emails)
# print(df_emails['summary'][id])

# SPACY - NER

In [ ]:
deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

In [ ]:
df_emails[['header','NER_header','NER_body']][20:45]



In [ ]:
id = 44
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")



In [ ]:
spacy.explain("PERSON")

In [ ]:
# A ce stade le fichier a la structure attendue par melusine + l'identification des NER par Spacy
df_emails[['header','NER_header','body','NER_body']].iloc[id]

In [ ]:
df_emails['body'].iloc[id]

# Summary

In [ ]:
df_emails[['body','body_dict']]